If you are just a student and dont have enough computation power and/or training set  
Just use a pre-trained model as we do  
Using pre-trained model from there: https://github.com/endernewton/tf-faster-rcnn  
Patching for using just CPU: https://github.com/endernewton/tf-faster-rcnn/issues/72  

In [1]:
import os 
import sys

this_dir = os.getcwd()
print(this_dir)

# Add lib to PYTHONPATH
lib_path = os.path.join(this_dir, 'tf-faster-rcnn', 'lib')
sys.path.insert(0, lib_path)
print(lib_path)

coco_path = os.path.join(this_dir, 'tf-faster-rcnn', 'data', 'coco', 'PythonAPI')
sys.path.insert(0, coco_path)

/home/jovyan/work/NetsLab
/home/jovyan/work/NetsLab/tf-faster-rcnn/lib


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from model.config import cfg
from model.test import im_detect
from model.nms_wrapper import nms

from utils.timer import Timer
import tensorflow as tf
import matplotlib.pyplot as plt
print(tf.__version__)
import numpy as np
import os, cv2
import argparse

from nets.vgg16 import vgg16
from nets.resnet_v1 import resnetv1

1.3.0


In [3]:
% matplotlib inline

In [4]:
anchor_scales = [2, 4, 8, 16, 32] 
CLASSES = ('background',
'person', 'bicycle', 'car', 'motorcycle', 'airplane',
'bus','train', 'truck', 'boat', 'traffic light', 'fire hydrant',
'stop sign', 'parking meter','bench', 'bird', 'cat', 'dog',
'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe',
'backpack','umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
'skis', 'snowboard', 'sports ball', 'kite','baseball bat', 'baseball glove',
'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
'fork', 'knife', 'spoon', 'bowl','banana', 'apple', 'sandwich', 'orange',
'broccoli','carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
'potted plant', 'bed', 'dining table','toilet', 'tv', 'laptop', 'mouse', 'remote',
'keyboard', 'cell phone', 'microwave', 'oven','toaster', 'sink', 'refrigerator',
'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush')


In [5]:
def vis_detections(class_name, dets, ax, thresh=0.5):
    """Draw detected bounding boxes."""
    inds = np.where(dets[:, -1] >= thresh)[0]
    if len(inds) == 0:
        return

    
    for i in inds:
        bbox = dets[i, :4]
        score = dets[i, -1]

        ax.add_patch(
            plt.Rectangle((bbox[0], bbox[1]),
                          bbox[2] - bbox[0],
                          bbox[3] - bbox[1], fill=False,
                          edgecolor='red', linewidth=3.5)
            )
        ax.text(bbox[0], bbox[1] - 2,
                '{:s} {:.3f}'.format(class_name, score),
                bbox=dict(facecolor='blue', alpha=0.5),
                fontsize=14, color='white')


    #plt.draw()
    #plt.savefig(save_path)
    #plt.show()

In [6]:
def demo(sess, net, image_name):
    """Detect object classes in an image using pre-computed object proposals."""

    # Load the demo image
    im_file = os.path.join(this_dir, "data/images", image_name)
    im_save_file = os.path.join(this_dir, "data/images_out/fasterrcnn_vgg", image_name)
    im = cv2.imread(im_file)

    # Detect all object classes and regress object bounds
    timer = Timer()
    timer.tic()
    scores, boxes = im_detect(sess, net, im)
    timer.toc()
    print('Detection took {:.3f}s for {:d} object proposals'.format(timer.total_time, boxes.shape[0]))

    # Visualize detections for each class
    CONF_THRESH = 0.8
    NMS_THRESH = 0.3
    
    im = im[:, :, (2, 1, 0)]
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(im, aspect='equal')
    for cls_ind, cls in enumerate(CLASSES[1:]):
        cls_ind += 1 # because we skipped background
        cls_boxes = boxes[:, 4*cls_ind:4*(cls_ind + 1)]
        cls_scores = scores[:, cls_ind]
        dets = np.hstack((cls_boxes,
                          cls_scores[:, np.newaxis])).astype(np.float32)
        keep = nms(dets, NMS_THRESH)
        dets = dets[keep, :]
        bbox = dets[:,:4]
        score = dets[:,-1]
        #print(cls)
        #print(bbox[score>CONF_THRESH], score[score>CONF_THRESH])
        vis_detections(cls, dets, ax, thresh=CONF_THRESH)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(im_save_file)
    plt.show()

In [ ]:
cfg.TEST.HAS_RPN = True  # Use RPN for proposals

# model path
tfmodel = os.path.join(this_dir, "data", "Faster-RCNN/vgg16_faster_rcnn_iter_1190000.ckpt")


if not os.path.isfile(tfmodel + '.meta'):
    raise IOError(('{:s} not found.\nDid you download the proper networks from '
                   'our server and place them properly?').format(tfmodel + '.meta'))

# init session
sess = tf.Session()
# load network
net = vgg16()
net.create_architecture("TEST", len(CLASSES), tag='default', anchor_scales=[4, 8, 16, 32])
saver = tf.train.Saver()
saver.restore(sess, tfmodel)

print('Loaded network {:s}'.format(tfmodel))

# im_names = ['000456.jpg', '000542.jpg', '001150.jpg',
#             '001763.jpg', '004545.jpg']
# for im_name in im_names:
#     print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
#     print('Demo for data/demo/{}'.format(im_name))
#     demo(sess, net, im_name)
# plt.show()

# Performance measurements block
import time
from tqdm import tqdm
perf_image = this_dir + "/data/images/" + "000542.jpg"
times = list()
im = cv2.imread(perf_image)
# for i in tqdm(range(100)):
#     timestart = time.time()
#     im_detect(sess, net, im)
#     times.append(time.time() - timestart)
# print(times)    
# performance = np.array(times)
# print("Mean:", np.mean(performance))
# print("Median:", np.median(performance))
# print("Q25:", np.percentile(performance, 25))
# print("Q75:", np.percentile(performance, 75))

sc, bx = im_detect(sess, net, im)
print(sc.shape, bx.shape)
exit()
##############################################################

########### Accuracy block ####################
import json
from tqdm import tqdm
ann_file = this_dir + '/data/annotations/instances_val2017.json'
test_images_path = this_dir + '/data/test_data/'
test_image_names = sorted(os.listdir(test_images_path))
cat_id_to_real_id = \
    {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16,
     18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 27: 25, 28: 26, 31: 27, 32: 28, 33: 29, 34: 30,
     35: 31, 36: 32, 37: 33, 38: 34, 39: 35, 40: 36, 41: 37, 42: 38, 43: 39, 44: 40, 46: 41, 47: 42, 48: 43, 49: 44,
     50: 45, 51: 46, 52: 47, 53: 48, 54: 49, 55: 50, 56: 51, 57: 52, 58: 53, 59: 54, 60: 55, 61: 56, 62: 57, 63: 58,
     64: 59, 65: 60, 67: 61, 70: 62, 72: 63, 73: 64, 74: 65, 75: 66, 76: 67, 77: 68, 78: 69, 79: 70, 80: 71, 81: 72,
     82: 73, 84: 74, 85: 75, 86: 76, 87: 77, 88: 78, 89: 79, 90: 80}
    
def parse_annotation(ann_file, img_dir, labels=[]):
    all_imgs = dict()
    seen_labels = set()
    with open(ann_file) as f:
        inst = json.load(f)
    boxes = {image['id']:dict() for image in inst['images']}
    cats = inst["categories"]
    for ann in inst['annotations']:
        obj = {}
        real_id = cat_id_to_real_id[ann['category_id']]
        namename = cats[real_id-1]["name"]
        seen_labels.add(namename)
        obj = []
        obj.append(int(round(ann['bbox'][1])))
        obj.append(int(round(ann['bbox'][0])))
        obj.append(int(round(ann['bbox'][1] + ann['bbox'][3])))        
        obj.append(int(round(ann['bbox'][0] + ann['bbox'][2])))
        if namename in boxes[ann["image_id"]]:
            boxes[ann['image_id']][namename].append(obj)
        else:
            boxes[ann['image_id']][namename] = [obj]
    for image in inst['images']:
        all_imgs[os.path.join(img_dir,image['file_name'])] =  boxes[image['id']]
    return all_imgs, seen_labels

imgs, labels = parse_annotation(ann_file, test_images_path)
        
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
 
    # compute the area of intersection rectangle
    interArea = (xB - xA + 1) * (yB - yA + 1)
 
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
 
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    if iou < 0:
        return 0
    # return the intersection over union value
    return iou

def compare_objdetect_accuracy(grrdict, predictdict):
    accs = []
    for key in grrdict:
        if key not in predictdict:
            accs += [0]* len(grrdict[key])
        else:
            for bbx in grrdict[key]:
                bbxaccs = []
                for bbox in predictdict[key]:
                    bbxaccs.append(bb_intersection_over_union(bbx, bbox))
                accs.append(max(bbxaccs))
            if len(grrdict[key]) < len(predictdict[key]):
                accs += [0]* (len(predictdict[key]) - len(grrdict[key]))
    for key in predictdict:
        if key not in grrdict:
            accs += [0]*len(predictdict[key])
    return np.mean(accs)

accuracys = []
for iname in tqdm(test_image_names):
    finame = test_images_path + iname
    print(iname)
    img = cv2.imread(finame)
    scores, boxes = im_detect(sess, net, img)
    prdctns = dict()
    for cls_ind, cls in enumerate(CLASSES[1:]):
        cls_ind += 1 # because we skipped background
        cls_boxes = boxes[:, 4*cls_ind:4*(cls_ind + 1)]
        cls_scores = scores[:, cls_ind]
        dets = np.hstack((cls_boxes,
                          cls_scores[:, np.newaxis])).astype(np.float32)
        keep = nms(dets, 0.3)
        dets = dets[keep, :]
        score = dets[:,-1]
        bbox = dets[:,:4][score>0.8]
        bbox = np.round(bbox).astype(int)
        if len(bbox) != 0:
            for bb in bbox:
                print(bb)
                if cls in prdctns:
                    prdctns[cls].append(bb)
                else:
                    prdctns[cls] = [bb]

    accuracys.append(compare_objdetect_accuracy(imgs[finame], prdctns))
print(accuracys)
print(np.mean(accuracys))
###############################################




sess.close()

INFO:tensorflow:Restoring parameters from /home/jovyan/work/NetsLab/data/Faster-RCNN/vgg16_faster_rcnn_iter_1190000.ckpt
Loaded network /home/jovyan/work/NetsLab/data/Faster-RCNN/vgg16_faster_rcnn_iter_1190000.ckpt
(103, 81) (103, 324)


  0%|          | 0/99 [00:00<?, ?it/s]

000000000139.jpg


  1%|          | 1/99 [00:07<12:09,  7.44s/it]

[419 158 469 295]
[383 172 405 212]
[165 233 186 265]
[293 219 353 315]
[366 218 440 313]
[231 174 266 213]
[  6 166 156 265]
000000000285.jpg


...

[137 200 183 275]
[103 178 467 372]
000000009772.jpg


 99%|█████████▉| 98/99 [10:46<00:06,  6.59s/it]

[275 231 329 350]
[131 557 416 630]
[423  92 515 186]
[327 379 403 399]
[176 379 256 397]
[  1 447  57 507]
000000009891.jpg


100%|██████████| 99/99 [10:52<00:00,  6.59s/it]

[ 79 100 205 292]
[196  97 349 409]
[307 113 353 154]
[358  88 639 308]
[171 241 309 403]
[425 233 494 336]
[3.6068274776212839, 0.69193521707093342, 0.20748155538807711, 0.090879728945972821, 0.12900516366787226, 0.067589688860637542, 0.2720517441412017, 0.17176304648139007, 2.2741793074468819, 0.15165115250036387, 2.3882269596060333, 0.12586609512220845, 0.15270961837542765, 0.0, 0.0, 0.078110198003736253, 0.14146680116370655, 3.2266494380873554, 0.24430365089913209, 0.086504470783946769, 0.40137031383102439, 0.91432192478520014, 0.69953808501216219, 0.15778054173576561, 0.38930042149752075, 0.17483655166732023, 0.00011195283054073218, 6.399723787757889, 0.073335917922913482, 15.744403710257728, 0.0059433372061846457, 0.0, 0.0018754850392342847, 0.083066560569881945, 0.057360609707180106, 0.19862395253724108, 1.8751103459780638, 0.20259298410064974, 0.3079388448644555, 0.05750333709826471, 0.75040747369235972, 0.15932239037219875, 4.3306049833866975, 0.15760937902647293, 0.1417941931

In [ ]:
path = this_dir + '/data/images/test_data'
image_names = sorted(os.listdir(path))